In [1]:
import os
import threading
import torch.multiprocessing as multiprocessing
import time
import functools

import sys
sys.path.insert(0, "../")

import matplotlib.pyplot as plt

import numpy as np
from tqdm import tqdm

import torch
from torch.optim import Adam
import scipy.stats as stats

from utils.kfp import get_B_block, diffusion_coeff, solve_pde, get_sparse_A_block, marginal_prob_std
from network.network import ScoreNet

from data.Dataset import CIFARDataset

torch.manual_seed(2);
torch.set_num_threads(1)

## Parameters

In [2]:
### Change these
N = 200
H = 28
W = 28
epochs = 3000
sigma = 250
lr = 1e-3
###

eps = 1e-6
dt = 1/N

In [3]:
marginal_prob_std_fn = functools.partial(marginal_prob_std, sigma=sigma)
diffusion_coeff_fn = functools.partial(diffusion_coeff, sigma=sigma)

## Instantiate Dataset

In [4]:
dataset = CIFARDataset(W, H);

channels = dataset.channels
n_data = len(dataset)

Files already downloaded and verified


## Create memory buffers


In [5]:
m = np.zeros((N, channels, H*W), dtype=np.float32)
m_prev = np.ones((N, channels, H*W), dtype=np.float32)
scores = np.zeros((N, channels, H*W), dtype=np.float32) # initial scores guess
dm = np.zeros_like(scores, dtype=np.float32)

# we want to sample from random time steps to construct training samples
time_ = np.linspace(eps, 1, N).astype(np.float32)

## Create training data

In [6]:
#@title Defining pde diffusion for multi-threading
def diffuse(x, m, dm, channel, time_, g, scores):

  y_train = []
  for j in range(H):
    y_train.append(x[channel][j, :])
  y_train = np.concatenate(y_train)
  x_train = []
  for l in range(W):
    x_train.append(x[channel][:, l])
  x_train = np.concatenate(x_train)

  xy_train = np.vstack([x_train, y_train])
  kde_kernel = stats.gaussian_kde(xy_train)
  m[0, channel] = kde_kernel.logpdf(xy_train)
  dx = 256/H*W
  
  sparse_A_block = get_sparse_A_block(dx, dt, g(time_[1:]), scores[1:, channel], H, W, N)

  B_block = get_B_block(dx, dt, m, channel, H, W, N)

  m[1:, channel] = solve_pde(sparse_A_block, B_block, mode='sp_sparse').reshape((-1, H*W))
  img_log_prob = m[:, channel]
  for i in range(N):
    dm[i, channel, 1:-1] = (img_log_prob[i,:-2] - img_log_prob[i,2:])/(2*dx)
    dm[i, channel, 0] = dm[i, channel, 1]
    dm[i, channel, -1] = dm[i, channel, -2]


In [7]:
res = 1
e = 0
tol = 1e-6
max_iter = 1000

while res > tol:
  for idx, data in tqdm(enumerate(dataset)):
    
    # diffuse all three channels concurrently
    for ch in range(channels):
        diffuse(data, m, dm, ch, time_, diffusion_coeff_fn, scores)

    scores = dm.copy()

    if e == max_iter:
      print(f'No convergence')
      break

    res = np.linalg.norm(m - m_prev)
    print(f'residual at iteration {e}: {res}')

    m_prev = m.copy()
    e += 1

scores_label = scores.copy().reshape((-1, channels, H, W))

0it [00:00, ?it/s]

1it [00:03,  3.98s/it]


residual at iteration 0: 588.0607299804688


1it [00:04,  4.29s/it]


residual at iteration 1: 187.83935546875


...

residual at iteration 3: 0.005180760752409697


1it [00:04,  4.26s/it]


residual at iteration 4: 3.15093930112198e-05


1it [00:04,  4.19s/it]

residual at iteration 5: 9.441224051442987e-07


## Train

In [8]:

def loss_fn(model, x, label, diffusion_coeff, marginal_prob_std, eps=1e-5):
  """The loss function for training score-based generative models.

  Args:
    model: A PyTorch model instance that represents a
      time-dependent score-based model.
    x: A mini-batch of training data.
    marginal_prob_std: A function that gives the standard deviation of
      the perturbation kernel.
    eps: A tolerance value for numerical stability.
  """
  random_t = torch.tensor(np.sort(np.random.uniform(eps, 1., N)).astype(np.float32))
  # we encode the label into the initial data using the reverse ODE
  diff_std2 = diffusion_coeff(2 * random_t)
  for i in range(1, N):
    x[i] = x[i-1] - 0.5 * label[i-1] * diff_std2[i-1] * dt
  std = marginal_prob_std(random_t)
  z = torch.randn_like(x)
  # we perturb the image by the forward SDE conditional distribution
  perturbed_x = x + z * std[:, None, None, None]
  score = model(perturbed_x, random_t)
  # loss = torch.mean(torch.sum((score * std[:, None, None, None] - label)**2, dim=(1, 2, 3)) / (2 * diff_std2))
  loss = torch.mean(torch.sum((score * std[:, None, None, None] + z)**2, dim=(1, 2, 3))) # original loss from tutorial
  return loss

In [9]:
#@title Function for training on a thread
def diffuse_train(channel, init_x, epoch, diffusion_coeff, marginal_prob_std, label):
  
  model_score = ScoreNet(marginal_prob_std=marginal_prob_std)
  optimizer = Adam(model_score.parameters(), lr=lr)
  model_score.train();

  scores_label = torch.tensor(label)[:, channel][:, None]
  for e in tqdm(range(epoch)):
    loss = loss_fn(model_score, init_x[:, channel][:, None], scores_label, diffusion_coeff, marginal_prob_std)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  print(f'\nloss at channel {channel}: {loss}')
  file = f'model_cifar_thread_{channel}.pth'
  torch.save(model_score.state_dict(), file)
  print(f"model for thread {channel} has been saved\n")
  
#@title Function for training on a thread
def diffuse_train_all_channels(init_x, epoch, diffusion_coeff, marginal_prob_std, label):
  
  model_score = ScoreNet(marginal_prob_std=marginal_prob_std)
  optimizer = Adam(model_score.parameters(), lr=lr)
  model_score.train();

  scores_label = torch.tensor(label)
  for e in tqdm(range(epoch)):
    loss = loss_fn(model_score, init_x, scores_label, diffusion_coeff, marginal_prob_std)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  print(f'\nloss at all channels: {loss}')
  file = f'model_cifar_thread_all.pth'
  torch.save(model_score.state_dict(), file)
  print(f"model for all channels has been saved\n")

#processes = [None] * channels
init_x = torch.zeros((N, channels, H, W))

diffuse_train_fn = functools.partial(diffuse_train, init_x=init_x, epoch=epochs, diffusion_coeff = diffusion_coeff_fn, marginal_prob_std = marginal_prob_std_fn, label=scores_label)

for idx, data in enumerate(dataset):
  for ch in range(channels):
    init_x[:, ch] = data[ch]
    
  diffuse_train_all_channels(init_x, epochs, diffusion_coeff_fn, marginal_prob_std_fn, scores_label)

  # # train all three channels concurrently
  # for ch in range(channels):
  #   processes[ch] = multiprocessing.Process(target=diffuse_train, args=[ch, init_x, epochs, diffusion_coeff_fn, marginal_prob_std_fn, scores_label])
  #   processes[ch].start()

  # for p in processes:
  #   processes.join()

  0%|          | 0/3000 [00:00<?, ?it/s]

  0%|          | 5/3000 [00:05<50:25,  1.01s/it]  


KeyboardInterrupt: 